In [1]:
#run this notebook in mlde2
from evcouplings.couplings import CouplingsModel
import pandas as pd
from evcouplings.mutate import predict_mutation_table, single_mutant_matrix
import numpy as np
import itertools

/home/jyang4/miniconda3/envs/mlde2/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/jyang4/miniconda3/envs/mlde2/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#d

In [2]:
len('GPDSMADITDKTAEQLENLNIQDDQKQAATGSESQSVENSSASLYVGDLEPSVSEAHLYDIFSPIGSVSSIRVCRDAITKTSLGYAYVNFNDHEAGRKAIEQLNYTPIKGRLCRIMWSQRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGQEIYVAPHLSRKERDSQLEETKAHYTNLYVKNINSETTDEQFQELFAKFGPIVSASLEKDADGKLKGFGFVNYEKHEDAVKAVEALNDSELNGEKLYVGRAQKKNERMHVLKKQYEAYRLEKMAKYQGVNLFVKNLDDSVDDEKLEEEFAPYGTITSAKVMRTENGKSKGFGFVCFSTPEEATKAITEKNQQIVAGKPLYVAIAQRKDVRRSQLAQQIQARNQMRYQQATAAAAAAAAGMPGQFMPPMFYGVMPPRGVPFNGPNPQQMNPMGGMPKNGMPPQFRNGPVYGVPPQGGFPRNANDNNQFYQQKQRQALGEQLYKKVSAKTSNEEAAGKITGMILDLPPQEVFPLLESDELFEQHYKEASAAYESFKKEQEQQTEQA')

581

In [71]:
#full PABP Sequence
str = 'MADITDKTAEQLENLNIQDDQKQAATGSESQSVENSSASLYVGDLEPSVSEAHLYDIFSPIGSVSSIRVCRDAITKTSLGYAYVNFNDHEAGRKAIEQLNYTPIKGRLCRIMWSQRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGKEIYVAPHLSRKERDSQLEETKAHYTNLYVKNINSETTDEQFQELFAKFGPIVSASLEKDADGKLKGFGFVNYEKHEDAVKAVEALNDSELNGEKLYVGRAQKKNERMHVLKKQYEAYRLEKMAKYQGVNLFVKNLDDSVDDEKLEEEFA'
#the indices listed by the CouplingsModel 122:204

In [2]:
#these should be the same model
#c1 is generated by the listgarten code
#c2 is provided in their data file
#c1 = CouplingsModel("uniref100.model_params")
#c2 = CouplingsModel("PABP_YEAST.model_params")

#model = CouplingsModel('/home/jyang4/combining-evolutionary-and-assay-labelled-data/inference/tm9D8/plmc/uniref100.model_params')

model = CouplingsModel('/home/jyang4/MLDE/code/PABP_data/PABP_YEAST.model_params')

In [4]:
#this files lists indices 114:204
data = pd.read_csv('/home/jyang4/ProtTP/data/PABP_AllPreds.csv', index_col=0)

In [63]:
data = pd.read_csv('/home/jyang4/ProtTP/data/All_DSM_PABP.csv', index_col=0)

/home/jyang4/miniconda3/envs/mlde2/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
#model.index_map.items()
#model.index_list
print(model.seq())

['K' 'G' 'S' 'G' 'N' 'I' 'F' 'I' 'K' 'N' 'L' 'H' 'P' 'D' 'I' 'D' 'N' 'K'
 'A' 'L' 'Y' 'D' 'T' 'F' 'S' 'V' 'F' 'G' 'D' 'I' 'L' 'S' 'S' 'K' 'I' 'A'
 'T' 'D' 'E' 'N' 'G' 'K' 'S' 'K' 'G' 'F' 'G' 'F' 'V' 'H' 'F' 'E' 'E' 'E'
 'G' 'A' 'A' 'K' 'E' 'A' 'I' 'D' 'A' 'L' 'N' 'G' 'M' 'L' 'L' 'N' 'G' 'Q'
 'E' 'I' 'Y' 'V' 'A' 'P' 'H' 'L' 'S' 'R']


In [65]:
def mutation_processing(model, file = '/home/jyang4/MLDE/code/trpB_data/tm9D8_SSM_muts.csv'):
    '''
    Adapted from Bruce.
    Inputs are the instance of the couplings model object and a list of mutations in the format [WTAA]_[position][newAA]+[WTAA]_[position][newAA].
    Output is all the EVMutation Scores
    '''
    #this file seems to have all the data but not sure what it is
    data = pd.read_csv(file, header = None)
    all_muts = data[0]
    # Define an array for storing outputs
    preds = np.empty(len(all_muts))

    # Loop over all combos
    for i, combo in enumerate(all_muts):

        # If the combo is wt, result is 0 and we can continue on
        # to the next
        if combo == None:
            preds[i] = 0
            continue

        # Build the mutation list. Don't include mutations that go back to wt
        muts = combo.split('+')
        mutation_list = [(int(mut[2:5]), mut[0], mut[5])
                        for mut in muts]
        mutation_list
        # Make the prediction
        preds[i], _, _ = model.delta_hamiltonian(mutation_list)
    return preds

#the functions below are adapted from the listgarten paper
def seq2mutation(seq, model, return_str=False, ignore_gaps=False,
        sep=":", offset=1):
    mutations = []
    for pf, pm in model.index_map.items():
        if seq[pm] != model.target_seq[pm]:
            if ignore_gaps and (
                    seq[pm] == '-' or seq[pm] not in model.alphabet):
                continue
            mutations.append((pf, model.target_seq[pm], seq[pm]))
    if return_str:
        return sep.join([m[1] + str(m[0]) + m[2] for m in mutations])
    return mutations

# def seq2mutation(seq, model, return_str=False, ignore_gaps=False,
#         sep=":", offset=1):
#     mutations = []
#     for pf, pm in model.index_map.items():
#         if seq[pf-offset] != model.target_seq[pm]:
#             if ignore_gaps and (
#                     seq[pf-offset] == '-' or seq[pf-offset] not in model.alphabet):
#                 continue
#             mutations.append((pf, model.target_seq[pm], seq[pf-offset]))
#     if return_str:
#         return sep.join([m[1] + str(m[0]) + m[2] for m in mutations])
#     return mutations

def seq2effect(seqs, model, offset=1, ignore_gaps=False):
    effects = np.zeros(len(seqs))
    for i in range(len(seqs)):
        #note the 8 here is specifcally for pabP
        mutations = seq2mutation(seqs[i][8:], model,
                ignore_gaps=ignore_gaps, offset=offset)
        dE, _, _ = model.delta_hamiltonian(mutations)
        effects[i] = dE
    return effects

In [17]:
preds = mutation_processing(model)

In [61]:
data['EVMutation'] = seq2effect(data['seq'].values,model)
data['EV_rank'] = data['EVMutation'].rank()
data.to_csv('PABP_AllPreds.csv')

In [66]:
data['EVMutation'] = seq2effect(data.iloc[:,0].values,model)

In [74]:
data = data.rename(columns = {'0': 'seq','EVMutations': 'EVMutation'})
data

,seq,EVMutation
0,QRDPSLRKAASGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-5.686316
1,QRDPSLRKACSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-8.104537
2,QRDPSLRKADSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-5.271788
3,QRDPSLRKAESGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-4.636597
4,QRDPSLRKAFSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-7.996856
...,...,...
1198876,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-8.542828
1198877,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-10.226447
1198878,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-11.471829
1198879,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-13.240669


In [75]:
data.to_csv('All_DSM_PABP.csv')

### The part after this is to get the mutations of the PABP Dataset in the format for triad

In [27]:
#rewrote this function so that it works to generate the PABP mutation file for triad
def seq2mutation(seq, model=model, return_str=False, ignore_gaps=False,
        sep=":", offset=1):
    mutations = []
    seq = seq[8:]
    for pf, pm in model.index_map.items():
        if seq[pm] != model.target_seq[pm]:
            if ignore_gaps and (
                    seq[pm] == '-' or seq[pm] not in model.alphabet):
                continue
            mutations.append((pm+9, model.target_seq[pm], seq[pm]))
    if return_str:
        return sep.join([m[1] + str(m[0]) + m[2] for m in mutations])
    return mutations

In [28]:
data['variants'] = data['seq'].apply(seq2mutation)
data

In [29]:
mutation_encodings = []

for  variant in data.variants.values:
    mut_encoding_list = []
    for mutation in variant:
        mut_encoding_list.append('A_' + str(mutation[0]) + mutation[2])
        # If the mut_encoding_list has no entries, continue (this is wild type)
    
    if len(mut_encoding_list) == 0:
        continue
    # Otherwise, append to mutation_encodings
    else:
        mutation_encodings.append("+".join(mut_encoding_list) + "\n") 

# Save the mutants
with open("PABP_RRM.mut", "w") as f:
    f.writelines(mutation_encodings)

### Reformatting the mutation in general:

In [6]:
#rewrote this function so that it works to generate the PABP mutation  in general
def seq2mutation(seq, model=model, return_str=False, ignore_gaps=False,
        sep=":", offset=1):
    mutations = []
    seq = seq[8:]
    for pf, pm in model.index_map.items():
        if seq[pm] != model.target_seq[pm]:
            if ignore_gaps and (
                    seq[pm] == '-' or seq[pm] not in model.alphabet):
                continue
            mutations.append((pf, model.target_seq[pm], seq[pm]))
    if return_str:
        return sep.join([m[1] + str(m[0]) + m[2] for m in mutations])
    return mutations

In [34]:
def mut2shorthand(muts):
    mut_encoding_list = []
    for mutation in muts:
        mut_encoding_list.append(str(mutation[1])+ str(mutation[0]) + mutation[2])

    if len(mut_encoding_list) == 0:
        code='WT'
    else:
        code = ":".join(mut_encoding_list) 
    return code

In [19]:
data['variants'] = data['seq'].apply(seq2mutation)
data

,seq,log_fitness,n_mut,EVMutation,EV_rank,variants
0,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,0.047837,1,-0.335980,37231.0,"[(194, Q, K)]"
1,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,0.037296,1,-2.912587,34905.0,"[(164, K, M)]"
2,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,0.024693,1,-1.090189,36841.0,"[(183, I, V)]"
3,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,-0.035006,1,2.899314,37700.0,"[(177, G, E)]"
4,QRDPSLRKKGSGNIFIKSLHPDIDNKALYDTFSVFGDILSSKIATD...,-0.063472,1,-7.358377,22888.0,"[(132, N, S)]"
...,...,...,...,...,...,...
37706,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILASKIATD...,-0.949072,2,-7.838261,21138.0,"[(154, S, A), (174, E, Q)]"
37707,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILASKIATD...,-3.453333,2,-17.776333,490.0,"[(154, S, A), (169, G, E)]"
37708,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDAYSVFGDILSSKIATD...,-0.322274,2,-11.215503,9534.0,"[(145, T, A), (146, F, Y)]"
37709,QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATD...,0.001000,2,-7.870227,21008.0,"[(171, V, A), (174, E, V)]"


In [36]:
data['mutant'] = data['variants'].apply(mut2shorthand)

In [41]:
data = data.drop(columns=["variants"])

In [43]:
data.to_csv('PABP_AllPreds.csv')

In [44]:
ESMdata = pd.read_csv('PABP_ESM.csv')

In [52]:
merged = data.merge(ESMdata[['mutant', 'mutation_effect_prediction_vae_ensemble']]).rename(columns={'mutation_effect_prediction_vae_ensemble': 'ESM_ensemble'})

In [53]:
merged.to_csv('PABP_AllPreds.csv')

### Code to generate all double mutant possibilities

In [48]:
ALL_AAS = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
WT = "QRDPSLRKKGSGNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGQEIYVAPHLSRKERDSQ"

In [49]:
#generate all the double mutants
all_combos = list(itertools.combinations(range(8,90), 2))
mutations = []
for pos1, pos2 in all_combos:
    aa1 = WT[pos1]
    aa2 = WT[pos2]
    choices1 = [x for x in ALL_AAS if x!= aa1]
    choices2 = [x for x in ALL_AAS if x!= aa2]
    for choice1 in choices1:
        for choice2 in choices2:
            variant = WT[:pos1] + choice1 + WT[pos1+1:]
            variant = variant[:pos2] + choice2 + variant[pos2+1:]
            mutations.append(variant)

In [55]:
df = pd.DataFrame(mutations)
df.to_csv('All_DSM_PABP.csv')